In [83]:
import pandas as pd # type: ignore

# Read the CSV file
data = pd.read_csv('train.csv')

# Display the first few rows of the data
data.head()
data.columns = data.columns.str.replace("'", "", regex=False)

In [84]:
def clean_beach_resort(dataset):
    allowed_values = [ 'Nulo' ,'Not specified', '0' , '1']
    data['Beach_Resort'] = data['Beach_Resort'].apply(lambda x: 'NA' if pd.isna(x) or x in allowed_values else x)
    return dataset

In [85]:
#data = clean_beach_resort(data)

In [86]:
#data.drop('Beach_Resort')

In [ ]:
def extract_date(data_set: pd.DataFrame) -> pd.DataFrame:
    #data_set['Publication_Date'] = data_set['Publication_Date'].str.extract(r'Publicado el (.+)')
    #data_set['Publication_Date'] = pd.to_datetime(data_set['Publication_Date'], errors='coerce')
    #data_set['Publication_Date'] = data_set['Publication_Date'].fillna(data_set['Publication_Date'].mode()[0])
    #data_set['Publication_Year'] = data_set['Publication_Date'].dt.year
    #data_set['Publication_Month'] = data_set['Publication_Date'].dt.month
    #data_set['Publication_Day'] = data_set['Publication_Date'].dt.day
    data_set.drop(columns='Publication_Date', inplace=True)
    return data_set

In [88]:
data = extract_date(data)

/tmp/ipykernel_36903/2842108097.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_set['Publication_Date'] = pd.to_datetime(data_set['Publication_Date'], errors='coerce')


In [89]:
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

In [90]:
data.groupby('Total_Area_m2').size()

Total_Area_m2
0                 43
1                  2
100               31
1000             143
1002               5
                ... 
99                 3
990                4
997                1
999                2
Not specified      1
Length: 1121, dtype: int64

In [91]:
for column in data.columns:
    print('--------------------------------')
    print(data.groupby(column).size())

--------------------------------
Id
0       1
1       1
2       1
3       1
4       1
       ..
6995    1
6996    1
6997    1
6998    1
6999    1
Length: 7000, dtype: int64
--------------------------------
Gas_Connection
0                3175
1                 484
Not specified    3341
dtype: int64
--------------------------------
Fireplace
0                3270
1                1319
Not specified    2411
dtype: int64
--------------------------------
Entrance_Hall
0                1864
1                2695
Not specified    2441
dtype: int64
--------------------------------
Kitchenette
0                6791
Not specified     209
dtype: int64
--------------------------------
Equipped
0                4161
1                 428
Not specified    2411
dtype: int64
--------------------------------
Garage_Type
Lineales          604
NoTiene           835
Not specified    2892
Paralelas        2505
Separadas         164
dtype: int64
--------------------------------
Furnished
0                3

In [92]:
# Remove Construction_Area column, its redundant with construction area
data.drop(columns=['Construction_Area'], inplace=True)


In [93]:
data['Total_Area_m2'] = pd.to_numeric(data['Total_Area_m2'], errors='coerce')

In [94]:
dict_bedrooms = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5+': 5
}
def preprocess_data(data_set):
    data_set['Bedrooms'] = data_set['Bedrooms'].apply(lambda x: dict_bedrooms.get(x, x))
    data_set['Bedrooms'].fillna(data_set['Bedrooms'].mode()[0], inplace=True)
    return data_set

In [95]:
data = preprocess_data(data)

/tmp/ipykernel_36903/1895079081.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_set['Bedrooms'].fillna(data_set['Bedrooms'].mode()[0], inplace=True)


In [96]:
numeric_columns = ['Number_Floors', 'Age']
def fill_numeric_column(column_name, dataset):
    dataset[column_name] = pd.to_numeric(dataset[column_name], errors='coerce')
    dataset[column_name].fillna(dataset[column_name].mode()[0], inplace=True)
    return dataset

In [97]:

for column in numeric_columns:
    data = fill_numeric_column(column, data)

/tmp/ipykernel_36903/1612466116.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column_name].fillna(dataset[column_name].mode()[0], inplace=True)


In [98]:
data

,Id,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,...,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Price,Publication_Year,Publication_Month,Publication_Day
0,0,1,Not specified,0,0,Not specified,Paralelas,Not specified,Not specified,0,...,NaN,NaN,NaN,NaN,NaN,NaN,210000,2019,9,18
1,1,0,1,Not specified,0,0,Paralelas,0,0,0,...,0,LaMolina,Not specified,Bueno,Not specified,1,660000,2019,4,9
2,2,Not specified,Not specified,1,0,Not specified,Lineales,Not specified,Not specified,Not specified,...,Not specified,LaMolina,0,Bueno,0,1,1500000,2019,1,8
3,3,0,1,Not specified,0,0,Separadas,0,0,0,...,0,SanIsidro,Not specified,Remodelado,Not specified,1,1100000,2019,12,8
4,4,0,Not specified,Not specified,0,Not specified,Lineales,Not specified,Not specified,0,...,Not specified,VillaMariaDelTriunfo,Not specified,Regular,Not specified,0,110000,2019,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,6995,Not specified,Not specified,Not specified,0,Not specified,Not specified,Not specified,Not specified,Not specified,...,Not specified,LaVictoria,Not specified,Bueno,Not specified,0,450000,2019,7,26
6996,6996,0,0,1,0,0,Not specified,1,0,0,...,0,Asia,0,Not specified,0,1,450000,2019,10,8
6997,6997,0,0,Not specified,0,0,Not specified,0,1,1,...,0,SantiagoDeSurco,Not specified,Not specified,Not specified,1,450000,2019,9,16
6998,6998,Not specified,Not specified,1,0,Not specified,Not specified,Not specified,Not specified,Not specified,...,Not specified,Bellavista,0,Muy bueno,0,1,240000,2019,8,29


In [99]:
allowed_values = ['0', '1', 'Nulo', 'Not specified', 'Si', 'No']
def remove_trash_data(column,dataset):
    print('--------------------------------')
    print('Removing trash data from column:', column)
    print(dataset.groupby(column).size())
    dataset = dataset[(dataset[column].isin(allowed_values)) | (dataset[column].isnull())]
    print(dataset.groupby(column).size())
    return dataset

In [100]:
trash_columns = ['Internet', 'Sauna_Area', 'Air_Conditioning', 'Independent_Entrance', 'Jacuzzi', 'Service_Bathroom'
                 ,'Cable', 'Service_Room', 'Internet_Room', 'Daycare', 'Daily_Dining_Room', 'Heating', 'Pets']
for column in trash_columns:
    data = remove_trash_data(column, data)

--------------------------------
Removing trash data from column: Internet
Internet
0                                                                2105
1                                                                1194
Not specified                                                    3160
Panamericana Sur Km 94.5 Asia                                       1
Panamericana Sur Km 94.5 Club Playa Las Arenas Asia                 1
Panamericana Sur Km 94.5 Club Playa Las Arenas Casa B_22 Asia       1
Publicado el 05.09.19                                               1
Publicado el 10.09.19                                               1
Publicado el 12.08.19                                               2
Publicado el 13.04.19                                               5
Publicado el 13.09.19                                               1
Publicado el 14.09.19                                               1
Publicado el 17.09.19                                               6
Public

In [101]:
data = data[data['Age']!= 'Not specified']
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

In [102]:
import numpy as np
def replace_not_specified_with_mode(column, dataset):
    dataset[column] = dataset[column].apply(lambda x: np.nan if (x == 'Not specified' or x == 'Nulo') else 
                                      True if (x == '1' or x == 'Si') 
                                      else False if (x == '0' or x == 'No') else x)
    #dataset[column+'_missing'] = dataset[column].isna().astype(int)
    dataset[column].fillna(dataset[column].mode()[0], inplace=True)
    #dataset[column].fillna(False, inplace=True)

    print(dataset.groupby(column).size())
    return dataset


In [103]:
# Posibles columnas repetidas en el dataset
# Sauna, Sauna_area,
# Oceanfront, Near_Sea
# Kitchenette, Kitchen_with_Cabinets


columns_replace_with_mode = ['Gas_Connection', 'Fireplace', 'Entrance_Hall', 'Kitchenette',
           'Equipped', 'Furnished', 'Drainage', 'Telephone', 'Guest_Bathroom', 'BBQ_Area',
           'Living_Room', 'Nearby_Parks', 'Solarium', 'Commercial_Use', 'Internal_Garden', 
           'Electricity', 'Patio', 'Children_Playground', 'Green_Areas', 'Electric_Doorman',
           'Intercom', 'Near_Sea', 'Sauna', 'Cinema_Room', 'Cleaning_Service', 'Terrace',
           'Sports_Area', 'Security_System', 'Water_Heater', 'Professional_Use', 'Club_House',
           'Internal_Park', 'Laundry_Room', 'Nearby_Schools', 'Balcony', 'Attic', 'Oceanfront', 'Security_Guard',
           'Swimming_Pool', 'Electric_Fence', 'Air_Conditioning', 'Hall', 'Nearby_Shopping_Centers', 'Kitchen',
           'Water', 'Basement', 'Independent_Bathroom', 'Walk_in_Closet', 'Grill', 'Closet',
           'Internet', 'Sauna_Area', 'Kitchen_with_Cabinets', 'Gym', 'Handicap_Access', 'Dining_Room',
           'Office', 'Service_Bathroom', 'Storage_Room', 'Cable', 'Jacuzzi', 'Independent_Entrance', 'Service_Room',
           'Internet_Room', 'Daycare', 'Daily_Dining_Room', 'Heating', 'Match', 'Pets', 'Garden']
for column in columns_replace_with_mode:
    print(f'Processing column: {column}')
    data = replace_not_specified_with_mode(column, data)

Processing column: Gas_Connection
Gas_Connection
False    6479
True      477
dtype: int64
Processing column: Fireplace
Fireplace
False    5645
True     1311
dtype: int64
Processing column: Entrance_Hall
Entrance_Hall
False    1856
True     5100
dtype: int64
Processing column: Kitchenette
Kitchenette
False    6956
dtype: int64
Processing column: Equipped
Equipped
False    6539
True      417
dtype: int64
Processing column: Furnished
Furnished
False    6143
True      813
dtype: int64
Processing column: Drainage
Drainage
False    5682
True     1274
dtype: int64
Processing column: Telephone
Telephone
False    5692
True     1264
dtype: int64
Processing column: Guest_Bathroom
Guest_Bathroom
False    3167
True     3789
dtype: int64
Processing column: BBQ_Area
BBQ_Area
False    6155
True      801
dtype: int64
Processing column: Living_Room
Living_Room
False    4042
True     2914
dtype: int64
Processing column: Nearby_Parks
Nearby_Parks
False     672
True     6284
dtype: int64
Processing column:

/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_3

Cinema_Room
False    6938
True       18
dtype: int64
Processing column: Cleaning_Service
Cleaning_Service
False    6565
True      391
dtype: int64
Processing column: Terrace
Terrace
False    3081
True     3875
dtype: int64
Processing column: Sports_Area
Sports_Area
False    6404
True      552
dtype: int64
Processing column: Security_System
Security_System
False    6009
True      947
dtype: int64
Processing column: Water_Heater
Water_Heater
False    5964
True      992
dtype: int64
Processing column: Professional_Use
Professional_Use
False     273
True     6683
dtype: int64
Processing column: Club_House
Club_House
False    6924
True       32
dtype: int64
Processing column: Internal_Park
Internal_Park
False    6840
True      116
dtype: int64
Processing column: Laundry_Room
Laundry_Room
False    2458
True     4498
dtype: int64
Processing column: Nearby_Schools
Nearby_Schools
False     718
True     6238
dtype: int64
Processing column: Balcony
Balcony
False    6097
True      859
dtype: int64

/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_3

Air_Conditioning
False    6170
True      786
dtype: int64
Processing column: Hall
Hall
False    2453
True     4503
dtype: int64
Processing column: Nearby_Shopping_Centers
Nearby_Shopping_Centers
False     672
True     6284
dtype: int64
Processing column: Kitchen
Kitchen
False     364
True     6592
dtype: int64
Processing column: Water
Water
False     748
True     6208
dtype: int64
Processing column: Basement
Basement
False    6646
True      310
dtype: int64
Processing column: Independent_Bathroom
Independent_Bathroom
False    6956
dtype: int64
Processing column: Walk_in_Closet
Walk_in_Closet
False    5430
True     1526
dtype: int64
Processing column: Grill
Grill
False    1987
True     4969
dtype: int64
Processing column: Closet
Closet
False    6630
True      326
dtype: int64
Processing column: Internet
Internet
False    5762
True     1194
dtype: int64
Processing column: Sauna_Area
Sauna_Area
False    6889
True       67
dtype: int64
Processing column: Kitchen_with_Cabinets
Kitchen_with_

/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_3

In [104]:
def clean_categorical_columns(dataset):
    categorical_cols = ['Garage_Type', 'Province', 'Type', 'Location', 'Natural_Light', 'Advertiser', 'Bedrooms', 'Pets', 'Garden', 'Beach_Resort', 'District', 'Property_Condition']
    for column in categorical_cols:
        print(f'Processing categorical column: {column}')
        dataset[column] = dataset[column].astype('category')
        dataset[column] = dataset[column].fillna(dataset[column].mode()[0])
    return dataset


In [105]:
data = clean_categorical_columns(data)

Processing categorical column: Garage_Type
Processing categorical column: Province
Processing categorical column: Type
Processing categorical column: Location
Processing categorical column: Natural_Light
Processing categorical column: Advertiser
Processing categorical column: Bedrooms
Processing categorical column: Pets
Processing categorical column: Garden
Processing categorical column: Beach_Resort
Processing categorical column: District
Processing categorical column: Property_Condition


In [106]:
# Drop unique columns
for column in data.columns:
    if len(data[column].unique()) == len(data):
        print(f'Dropping unique column: {column}')
        data.drop(columns=[column], inplace=True)
    elif len(data[column].unique()) == 1:
        print(f'Dropping column with one value: {column}')
        data.drop(columns=[column], inplace=True)

Dropping unique column: Id


Dropping column with one value: Kitchenette
Dropping column with one value: Match
Dropping column with one value: Independent_Bathroom
Dropping column with one value: Independent_Entrance


In [107]:
data

,Gas_Connection,Fireplace,Entrance_Hall,Equipped,Garage_Type,Furnished,Drainage,Telephone,Guest_Bathroom,BBQ_Area,...,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Price,Publication_Year,Publication_Month,Publication_Day
0,True,False,False,False,Paralelas,False,False,False,False,False,...,False,LaMolina,False,Not specified,False,True,210000,2019,9,18
1,False,True,True,False,Paralelas,False,False,False,True,False,...,False,LaMolina,False,Bueno,False,True,660000,2019,4,9
2,False,False,True,False,Lineales,False,False,False,False,False,...,False,LaMolina,False,Bueno,False,True,1500000,2019,1,8
3,False,True,True,False,Separadas,False,False,False,True,False,...,False,SanIsidro,False,Remodelado,False,True,1100000,2019,12,8
4,False,False,True,False,Lineales,False,False,False,False,False,...,False,VillaMariaDelTriunfo,False,Regular,False,False,110000,2019,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,False,False,True,False,Not specified,False,False,False,False,False,...,False,LaVictoria,False,Bueno,False,False,450000,2019,7,26
6996,False,False,True,False,Not specified,True,False,False,True,False,...,False,Asia,False,Not specified,False,True,450000,2019,10,8
6997,False,False,True,False,Not specified,False,True,True,True,False,...,False,SantiagoDeSurco,False,Not specified,False,True,450000,2019,9,16
6998,False,False,True,False,Not specified,False,False,False,True,False,...,False,Bellavista,False,Muy bueno,False,True,240000,2019,8,29


In [108]:
# Identify categorical and numerical columns
columns_object =data.select_dtypes(include=['object']).columns.tolist()
print(columns_object)
data.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()
data.drop(columns=columns_object, inplace=True)

[]


In [109]:
data.select_dtypes(include=['int64', 'float64']).columns.tolist()

['Garages',
 'Number_Bathrooms',
 'Construction_Area_m2',
 'Age',
 'Number_Floors',
 'Total_Area_m2',
 'Price']

In [110]:
import numpy as np
def replace_not_specified_with_mediam(column, dataset):
    dataset[column] = dataset[column].apply(lambda x: np.nan if (x == 'Not specified' or x == 'Nulo') else x)
    dataset[column].fillna(dataset[column].median(), inplace=True)
    return dataset

In [111]:
data = replace_not_specified_with_mediam('Total_Area_m2', data)
data.drop_duplicates(inplace=True)

/tmp/ipykernel_36903/4007228366.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].median(), inplace=True)


In [ ]:
data.drop(columns=['Advertiser', 'Cleaning_Service', 'Telephone', 'Internet_Room', 'Service_Bathroom', 'Service_Room', 'Handicap_Access', 'Office', 'Club_House', 'Kitchen'], inplace=True)

--------------------------------
Gas_Connection
False    6445
True      476
dtype: int64
--------------------------------
Fireplace
False    5616
True     1305
dtype: int64
--------------------------------
Entrance_Hall
False    1849
True     5072
dtype: int64
--------------------------------
Equipped
False    6505
True      416
dtype: int64
--------------------------------
Garage_Type
Lineales          589
NoTiene           831
Not specified    2870
Paralelas        2471
Separadas         160
dtype: int64
--------------------------------
Furnished
False    6116
True      805
dtype: int64
--------------------------------
Drainage
False    5649
True     1272
dtype: int64
--------------------------------
Telephone
False    5659
True     1262
dtype: int64
--------------------------------
Guest_Bathroom
False    3146
True     3775
dtype: int64
--------------------------------
BBQ_Area
False    6124
True      797
dtype: int64
--------------------------------
Living_Room
False    4028
True  

/tmp/ipykernel_36903/4175853395.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(data.groupby(column).size())
/tmp/ipykernel_36903/4175853395.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(data.groupby(column).size())
/tmp/ipykernel_36903/4175853395.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(data.groupby(column).size())
/tmp/ipykernel_36903/4175853395.py:3: FutureWarning: The default of

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Select features and target
X = data.drop(columns=['Price'])
y = data['Price']
y = np.log(y)  # Log-transform the target variable for better performance

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['category']).columns.tolist()
numerical_cols = ['Garages', 'Number_Bathrooms', 'Age', 'Number_Floors']
log_cols = ['Construction_Area_m2', 'Total_Area_m2']
boolean_cols = X.select_dtypes(include=['bool']).columns.tolist()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('bool', OneHotEncoder() ,boolean_cols),
        ('num', StandardScaler(), numerical_cols),
        ('log', FunctionTransformer(np.log), log_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit and transform data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Build neural network
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_processed.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


optimizer = Adam(clipvalue=0.001)  # Clip gradients to a value between -1 and 1
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train_processed, y_train, epochs=150, batch_size=64, validation_split=0.1, callbacks=[early_stop], verbose=1)

# Evaluate model
loss, mae = model.evaluate(X_test_processed, y_test, verbose=0)
print(f"Test MAE: {mae:.2f}")

/home/noah/Projects/PUCP_machine-learning_garment-productivity/myenv/lib/python3.13/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 22.6203 - mae: 3.2366 - val_loss: 0.9793 - val_mae: 0.8949
Epoch 2/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.5366 - mae: 0.6091 - val_loss: 0.6271 - val_mae: 0.7030
Epoch 3/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.4441 - mae: 0.5442 - val_loss: 0.2334 - val_mae: 0.3712
Epoch 4/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.2040 - mae: 0.3408 - val_loss: 0.1645 - val_mae: 0.3085
Epoch 5/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.3365 - mae: 0.4623 - val_loss: 0.1760 - val_mae: 0.3232
Epoch 6/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.2946 - mae: 0.4364 - val_loss: 0.2528 - val_mae: 0.3954
Epoch 7/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.2883 - mae: 0.4392 - val_loss: 0.4261 - val_mae: 0.5698
Epoch 8/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.2826 - mae: 0.4304 - val_loss: 0.2981 - val_mae: 0.4399
Epoch 9/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - l

In [114]:
'''
import tensorflow as tf
from tensorflow.keras import layers, models
from xgboost import XGBRegressor

input_dim = X_train_processed.shape[1]
encoding_dim = 20  # compress to 4 features

# Build autoencoder model
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(8, activation='relu')(input_layer)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)  # bottleneck

decoded = layers.Dense(8, activation='relu')(encoded)
decoded = layers.Dense(input_dim, activation='linear')(decoded)

autoencoder = models.Model(inputs=input_layer, outputs=decoded)

# Compile and train
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train_processed, X_train_processed, epochs=100, batch_size=32, shuffle=True, validation_data=(X_test_processed, X_test_processed), verbose=0)
'''

"\nimport tensorflow as tf\nfrom tensorflow.keras import layers, models\nfrom xgboost import XGBRegressor\n\ninput_dim = X_train_processed.shape[1]\nencoding_dim = 20  # compress to 4 features\n\n# Build autoencoder model\ninput_layer = layers.Input(shape=(input_dim,))\nencoded = layers.Dense(8, activation='relu')(input_layer)\nencoded = layers.Dense(encoding_dim, activation='relu')(encoded)  # bottleneck\n\ndecoded = layers.Dense(8, activation='relu')(encoded)\ndecoded = layers.Dense(input_dim, activation='linear')(decoded)\n\nautoencoder = models.Model(inputs=input_layer, outputs=decoded)\n\n# Compile and train\nautoencoder.compile(optimizer='adam', loss='mse')\nautoencoder.fit(X_train_processed, X_train_processed, epochs=100, batch_size=32, shuffle=True, validation_data=(X_test_processed, X_test_processed), verbose=0)\n"

In [115]:
'''
encoder = models.Model(inputs=input_layer, outputs=encoded)

# Encode training and test data
X_train_encoded = encoder.predict(X_train_processed)
X_test_encoded = encoder.predict(X_test_processed)
'''

'\nencoder = models.Model(inputs=input_layer, outputs=encoded)\n\n# Encode training and test data\nX_train_encoded = encoder.predict(X_train_processed)\nX_test_encoded = encoder.predict(X_test_processed)\n'

In [116]:

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
# Create the model
mlp = MLPRegressor(hidden_layer_sizes=(50, 30), max_iter=10000, random_state=1)

# Train the model
mlp.fit(X_train_processed, y_train)
y_pred = mlp.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

R2 score on test set: 0.7490


In [117]:
import numpy as np
from sklearn.linear_model import Lasso

# Create Lasso model with regularization strength (alpha)
lasso = Lasso(alpha=0.1)

# Fit to training data
lasso.fit(X_train_processed, y_train)

y_pred = lasso.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

R2 score on test set: 0.5962


In [118]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=10,
    random_state=151
)

# Train the model
xgb_model.fit(X_train_processed, y_train)

y_pred = xgb_model.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

R2 score on test set: 0.8125


In [119]:
from sklearn.metrics import r2_score

# Predict on test set
y_pred = model.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R2 score on test set: 0.7615


In [120]:
test_data = pd.read_csv('test.csv')
test_data.columns = test_data.columns.str.replace("'", "", regex=False)
test_data.head()



,Id,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,...,Handicap_Access,Dining_Room,Office,Total_Area_m2,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom
0,0,Not specified,0,1,0,0,Paralelas,0,0,Not specified,...,Not specified,1,1,340,0,LaMolina,0,Not specified,0,1
1,1,Not specified,Not specified,1,0,Not specified,Lineales,Not specified,Not specified,Not specified,...,Not specified,1,0,1138,Not specified,LaMolina,0,Bueno,0,1
2,2,Not specified,0,0,0,0,Paralelas,0,0,Not specified,...,Not specified,0,1,1353,0,LaMolina,0,Bueno,0,1
3,3,0,1,0,0,0,Paralelas,0,0,0,...,Not specified,1,0,230,0,SantiagoDeSurco,0,Muy bueno,0,1
4,4,0,Not specified,Not specified,0,Not specified,Not specified,Not specified,Not specified,0,...,Not specified,1,0,305,Not specified,LaMolina,Not specified,Not specified,Not specified,1


In [ ]:
# Remove Construction_Area column, its redundant with construction area
#test_data.drop(columns=['Beach:Resort'], inplace=True)
test_data = extract_date(test_data)
#test_data = clean_beach_resort(test_data)
test_data.drop(columns=['Construction_Area'], inplace=True)
test_data['Total_Area_m2'] = pd.to_numeric(test_data['Total_Area_m2'], errors='coerce')
test_data = replace_not_specified_with_mediam('Total_Area_m2', test_data)
test_data = preprocess_data(test_data)
for column in numeric_columns:
    test_data = fill_numeric_column(column, test_data)

test_data['Age'] = pd.to_numeric(test_data['Age'], errors='coerce')

for column in trash_columns:
    test_data = remove_trash_data(column, test_data)
for column in columns_replace_with_mode:
    print(f'Processing column: {column}')
    test_data = replace_not_specified_with_mode(column, test_data)

test_data = clean_categorical_columns(test_data)
    
# Drop unique columns
for column in test_data.columns:
    if len(test_data[column].unique()) == len(test_data):
        print(f'Dropping unique column: {column}')
        test_data.drop(columns=[column], inplace=True)
    elif len(test_data[column].unique()) == 1:
        print(f'Dropping column with one value: {column}')
        test_data.drop(columns=[column], inplace=True)

test_data.drop(columns=columns_object, inplace=True)
test_data.drop(columns=['Advertiser', 'Cleaning_Service', 'Telephone', 'Internet_Room', 'Service_Bathroom', 'Service_Room', 'Handicap_Access', 'Office', 'Club_House', 'Kitchen'], inplace=True)
test_data.info()

--------------------------------

/tmp/ipykernel_36903/2842108097.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_set['Publication_Date'] = pd.to_datetime(data_set['Publication_Date'], errors='coerce')
/tmp/ipykernel_36903/4007228366.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].median(), inplace=True)
/tmp/ipykernel_36903/1895079081.py:11: FutureWarning: A value is try


Removing trash data from column: Internet
Internet
0                192
1                112
Not specified    309
dtype: int64
Internet
0                192
1                112
Not specified    309
dtype: int64
--------------------------------
Removing trash data from column: Sauna_Area
Sauna_Area
0                404
1                  8
Not specified    201
dtype: int64
Sauna_Area
0                404
1                  8
Not specified    201
dtype: int64
--------------------------------
Removing trash data from column: Air_Conditioning
Air_Conditioning
0                317
1                 72
Not specified    224
dtype: int64
Air_Conditioning
0                317
1                 72
Not specified    224
dtype: int64
--------------------------------
Removing trash data from column: Independent_Entrance
Independent_Entrance
0                249
Not specified    364
dtype: int64
Independent_Entrance
0                249
Not specified    364
dtype: int64
----------------------------

/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_36903/3288349312.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[column].fillna(dataset[column].mode()[0], inplace=True)
/tmp/ipykernel_3

Club_House
False    655
True       2
dtype: int64
Processing column: Internal_Park
Internal_Park
False    641
True      16
dtype: int64
Processing column: Laundry_Room
Laundry_Room
False    225
True     432
dtype: int64
Processing column: Nearby_Schools
Nearby_Schools
False     65
True     592
dtype: int64
Processing column: Balcony
Balcony
False    578
True      79
dtype: int64
Processing column: Attic
Attic
False    637
True      20
dtype: int64
Processing column: Oceanfront
Oceanfront
False    625
True      32
dtype: int64
Processing column: Security_Guard
Security_Guard
False    523
True     134
dtype: int64
Processing column: Swimming_Pool
Swimming_Pool
False    452
True     205
dtype: int64
Processing column: Electric_Fence
Electric_Fence
False    623
True      34
dtype: int64
Processing column: Air_Conditioning
Air_Conditioning
False    585
True      72
dtype: int64
Processing column: Hall
Hall
False    236
True     421
dtype: int64
Processing column: Nearby_Shopping_Centers
Nea

In [122]:
test_data

,Gas_Connection,Fireplace,Entrance_Hall,Equipped,Garage_Type,Furnished,Drainage,Telephone,Guest_Bathroom,BBQ_Area,...,Total_Area_m2,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Publication_Year,Publication_Month,Publication_Day
0,False,False,True,False,Paralelas,False,False,False,False,False,...,340.0,False,LaMolina,False,Not specified,False,True,2019,8,20
1,False,False,True,False,Lineales,False,False,False,True,False,...,1138.0,False,LaMolina,False,Bueno,False,True,2019,8,22
2,False,False,False,False,Paralelas,False,False,False,False,False,...,1353.0,False,LaMolina,False,Bueno,False,True,2019,9,16
3,False,True,False,False,Paralelas,False,False,False,False,False,...,230.0,False,SantiagoDeSurco,False,Muy bueno,False,True,2019,8,20
4,False,False,True,False,Not specified,False,False,False,True,False,...,305.0,False,LaMolina,False,Not specified,False,True,2019,9,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,False,False,True,False,Paralelas,False,False,False,False,False,...,600.0,False,LaMolina,False,Not specified,False,True,2019,8,9
653,True,False,True,True,Separadas,True,False,False,True,True,...,204.0,False,Asia,False,Excelente,False,True,2019,8,29
654,False,False,True,False,Paralelas,False,False,False,True,False,...,270.0,False,LaMolina,False,Not specified,False,True,2019,8,9
655,False,False,False,False,Paralelas,False,False,False,False,True,...,160.0,False,Asia,False,Not specified,False,True,2019,7,25


In [123]:
# Identify categorical and numerical columns
#columns_object =test_data.select_dtypes(include=['object']).columns.tolist()
print(columns_object)
data.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()
#test_data.drop(columns=columns_object, inplace=True)

[]


['Gas_Connection',
 'Fireplace',
 'Entrance_Hall',
 'Equipped',
 'Furnished',
 'Drainage',
 'Telephone',
 'Guest_Bathroom',
 'BBQ_Area',
 'Living_Room',
 'Nearby_Parks',
 'Solarium',
 'Commercial_Use',
 'Internal_Garden',
 'Garages',
 'Electricity',
 'Patio',
 'Children_Playground',
 'Number_Bathrooms',
 'Green_Areas',
 'Electric_Doorman',
 'Construction_Area_m2',
 'Intercom',
 'Near_Sea',
 'Sauna',
 'Cinema_Room',
 'Cleaning_Service',
 'Terrace',
 'Sports_Area',
 'Security_System',
 'Water_Heater',
 'Professional_Use',
 'Club_House',
 'Internal_Park',
 'Laundry_Room',
 'Nearby_Schools',
 'Balcony',
 'Attic',
 'Oceanfront',
 'Security_Guard',
 'Swimming_Pool',
 'Electric_Fence',
 'Air_Conditioning',
 'Hall',
 'Nearby_Shopping_Centers',
 'Kitchen',
 'Water',
 'Basement',
 'Walk_in_Closet',
 'Age',
 'Number_Floors',
 'Daily_Dining_Room',
 'Grill',
 'Closet',
 'Internet',
 'Sauna_Area',
 'Kitchen_with_Cabinets',
 'Service_Room',
 'Storage_Room',
 'Cable',
 'Gym',
 'Jacuzzi',
 'Handicap_Ac

In [124]:
data.columns

Index(['Gas_Connection', 'Fireplace', 'Entrance_Hall', 'Equipped',
       'Garage_Type', 'Furnished', 'Drainage', 'Telephone', 'Guest_Bathroom',
       'BBQ_Area', 'Living_Room', 'Nearby_Parks', 'Solarium', 'Commercial_Use',
       'Province', 'Internal_Garden', 'Garages', 'Electricity', 'Patio',
       'Children_Playground', 'Type', 'Number_Bathrooms', 'Green_Areas',
       'Electric_Doorman', 'Construction_Area_m2', 'Intercom', 'Near_Sea',
       'Sauna', 'Cinema_Room', 'Cleaning_Service', 'Terrace', 'Sports_Area',
       'Security_System', 'Location', 'Water_Heater', 'Professional_Use',
       'Club_House', 'Internal_Park', 'Laundry_Room', 'Nearby_Schools',
       'Balcony', 'Attic', 'Oceanfront', 'Security_Guard', 'Natural_Light',
       'Swimming_Pool', 'Electric_Fence', 'Advertiser', 'Air_Conditioning',
       'Hall', 'Nearby_Shopping_Centers', 'Kitchen', 'Water', 'Basement',
       'Walk_in_Closet', 'Age', 'Number_Floors', 'Daily_Dining_Room', 'Grill',
       'Closet', 'Internet

In [125]:
test_data

,Gas_Connection,Fireplace,Entrance_Hall,Equipped,Garage_Type,Furnished,Drainage,Telephone,Guest_Bathroom,BBQ_Area,...,Total_Area_m2,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Publication_Year,Publication_Month,Publication_Day
0,False,False,True,False,Paralelas,False,False,False,False,False,...,340.0,False,LaMolina,False,Not specified,False,True,2019,8,20
1,False,False,True,False,Lineales,False,False,False,True,False,...,1138.0,False,LaMolina,False,Bueno,False,True,2019,8,22
2,False,False,False,False,Paralelas,False,False,False,False,False,...,1353.0,False,LaMolina,False,Bueno,False,True,2019,9,16
3,False,True,False,False,Paralelas,False,False,False,False,False,...,230.0,False,SantiagoDeSurco,False,Muy bueno,False,True,2019,8,20
4,False,False,True,False,Not specified,False,False,False,True,False,...,305.0,False,LaMolina,False,Not specified,False,True,2019,9,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,False,False,True,False,Paralelas,False,False,False,False,False,...,600.0,False,LaMolina,False,Not specified,False,True,2019,8,9
653,True,False,True,True,Separadas,True,False,False,True,True,...,204.0,False,Asia,False,Excelente,False,True,2019,8,29
654,False,False,True,False,Paralelas,False,False,False,True,False,...,270.0,False,LaMolina,False,Not specified,False,True,2019,8,9
655,False,False,False,False,Paralelas,False,False,False,False,True,...,160.0,False,Asia,False,Not specified,False,True,2019,7,25


In [126]:
test_data_processed = preprocessor.transform(test_data)
# Predict prices
predicted_prices = xgb_model.predict(test_data_processed)
print(predicted_prices)
predicted_prices = np.exp(predicted_prices)  # Inverse log transformation to get actual prices
# Add predictions to test_data
test_data['Predicted_Price'] = predicted_prices.flatten()
test_data[['Predicted_Price']].head()

df = pd.DataFrame(predicted_prices, columns=['Price'])
df.index.name = 'Id'
print(df)
df.to_csv('submission.csv', index=True)

[13.57896   14.091348  14.242217  13.035256  13.375172  12.442023
 13.554833  13.896888  12.01688   13.27155   13.476052  12.297232
 13.421438  14.145245  14.394049  13.811791  13.14833   13.643996
 13.266011  11.760066  13.401101  12.509612  13.623326  13.350559
 12.8219385 14.095508  12.550331  13.568673  13.090615  12.329385
 12.651914  13.33545   12.947504  12.950459  12.547437  14.065584
 14.375187  12.339268  13.138418  13.146523  12.770046  12.287425
 12.783195  11.486948  13.875842  13.347086  13.378844  13.728947
 14.164027  12.106921  14.09402   11.637123  13.906758  14.715524
 12.054852  12.75853   14.353582  13.977145  12.699121  13.210129
 13.45457   12.976514  12.507805  14.443512  14.202302  14.06245
 13.307807  14.061271  12.987949  13.227646  14.20724   12.755947
 14.207435  13.174255  13.236135  11.835376  14.001043  14.290429
 12.405776  11.760947  14.090054  12.446276  13.240503  13.423544
 12.0852995 12.583042  14.338388  11.496988  14.0776005 13.130143
 13.227064 